In [17]:
import algorithms
from importlib import reload
reload(algorithms)
from algorithms import *

filename = "test"
input_file = f'sync_data/{filename}.csv'
output_file = f'output/{filename}_communities.csv'

infomap_communities(input_file, output_file, jsd_relax_rate=0.15)

In [18]:
import measurement
reload(measurement)
from measurement import *

compute_layerwise_ami_avg("test","sync_data",output_dir="output")

[0.8641253736288659, 0.9336986067992395, 0.8876319653726334, 0.8943528197602778, 0.9082457389242938]


0.8976109008970623

In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Build a multilayer network from a CSV with columns:
    u,v,layer,u_label,v_label
Compute Node2Vec embeddings per layer, then compute pairwise
Gromov–Wasserstein (GW) distances between layers using POT.

No argparse version: set your parameters in the CONFIG section below
and run the script directly (python multilayer_node2vec_gw.py).

Dependencies (install if needed):
    pip install pandas networkx node2vec POT gensim

Outputs:
  - out/embeddings_layer_<layer>.csv : per-layer embeddings
  - out/gw_distances.csv             : symmetric matrix of GW distances between layers
"""

from pathlib import Path
from typing import Dict, List
import os

import numpy as np
import pandas as pd
import networkx as nx

# =====================
# CONFIG — EDIT HERE
# =====================
CSV_PATH = "sync_data/test.csv"          # 输入 CSV 路径
OUT_DIR = "out"                           # 输出目录

# node2vec 超参数
DIMENSIONS = 64
WALK_LENGTH = 80
NUM_WALKS = 10
P = 1.0
Q = 1.0
WINDOW = 10
WORKERS = 1
SEED = 42

# GW 距离使用的“空间”
#   - 'embedding' : 用每层的 node2vec 向量计算层内两两节点距离矩阵，再做 GW（你想要的）
#   - 'shortest_path' : 用图的最短路距离做 GW（原实现，作为可选）
GW_SPACE = 'embedding'   # 'embedding' 或 'shortest_path'
GW_EMB_METRIC = 'euclidean'  # 'euclidean' 或 'cosine'
# =====================
# IMPLEMENTATION
# =====================

# node2vec implementation
try:
    from node2vec import Node2Vec
except Exception as e:
    raise SystemExit(
        "The 'node2vec' package is required. Install via: pip install node2vec gensim"
        f"Import error: {e}"
    )

# POT for GW distance
try:
    from ot.gromov import gromov_wasserstein2
except Exception as e:
    raise SystemExit(
        "The 'POT' package is required. Install via: pip install POT"
        f"Import error: {e}"
    )


def read_edges(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    required = {"u", "v", "layer"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV missing required columns: {sorted(missing)}")
    return df


def build_layer_graphs(df: pd.DataFrame) -> Dict[str, nx.Graph]:
    """Return dict: layer -> NetworkX Graph built from edges in that layer."""
    graphs: Dict[str, nx.Graph] = {}
    for layer, gdf in df.groupby("layer"):
        G = nx.Graph()
        edges = [(str(u), str(v)) for u, v in zip(gdf["u"], gdf["v"])]
        G.add_edges_from(edges)
        for col in ["u", "v"]:
            for n in gdf[col].astype(str).unique():
                if n not in G:
                    G.add_node(n)
        graphs[str(layer)] = G
    return graphs


def compute_node2vec_embeddings(
    G: nx.Graph,
    dimensions: int = 64,
    walk_length: int = 80,
    num_walks: int = 10,
    p: float = 1.0,
    q: float = 1.0,
    window: int = 10,
    workers: int = 4,
    seed: int = 42,
) -> pd.DataFrame:
    if len(G) == 0:
        raise ValueError("Graph has no nodes.")

    def _fit(n_jobs: int):
        n2v = Node2Vec(
            G,
            dimensions=dimensions,
            walk_length=walk_length,
            num_walks=num_walks,
            p=p,
            q=q,
            workers=n_jobs,   # used both for walk generation (joblib) and gensim training threads
            seed=seed,
            quiet=True,
        )
        return n2v.fit(window=window, min_count=1, batch_words=4)

    try:
        model = _fit(workers)
    except Exception as e:
        # Fallback for environments where joblib multiprocessing breaks (e.g., numpy/joblib mismatch)
        print(f"[WARN] node2vec parallel walk generation failed with workers={workers} due to: {e}"
              f"       Falling back to single-process mode (workers=1). Consider upgrading numpy/joblib.")
        model = _fit(1)

    nodes = [str(n) for n in G.nodes()]
    vecs = np.vstack([model.wv[n] for n in nodes])
    cols = [f"f{i}" for i in range(dimensions)]
    emb = pd.DataFrame(vecs, index=nodes, columns=cols)
    emb.index.name = "node"
    return emb


def _all_pairs_shortest_path_matrix(G: nx.Graph, nodes: List[str]) -> np.ndarray:
    spl = dict(nx.all_pairs_shortest_path_length(G))
    finite_dists = []
    for i in nodes:
        di = spl.get(i, {})
        for j, d in di.items():
            if i != j:
                finite_dists.append(d)
    max_finite = max(finite_dists) if finite_dists else 0
    penalty = (max_finite + 1) if max_finite > 0 else 1.0

    n = len(nodes)
    C = np.zeros((n, n), dtype=float)
    for a, i in enumerate(nodes):
        di = spl.get(i, {})
        for b, j in enumerate(nodes):
            if i == j:
                C[a, b] = 0.0
            else:
                C[a, b] = float(di.get(j, penalty))
    return C


def _pairwise_distance_matrix_from_embeddings(emb: pd.DataFrame, metric: str = 'euclidean') -> np.ndarray:
    """Compute layer-internal pairwise distances from node embeddings.
    emb: DataFrame indexed by node id, columns are features.
    metric: 'euclidean' or 'cosine' (converted to distance: 1 - cosine).
    """
    X = emb.values.astype(float)
    if metric == 'euclidean':
        # (x - y)^2 = ||x||^2 + ||y||^2 - 2 x·y
        sq_norms = np.sum(X * X, axis=1, keepdims=True)
        # broadcasting to get squared distances
        D2 = sq_norms + sq_norms.T - 2 * (X @ X.T)
        np.maximum(D2, 0.0, out=D2)
        return np.sqrt(D2, dtype=float)
    elif metric == 'cosine':
        # normalize rows
        norms = np.linalg.norm(X, axis=1, keepdims=True)
        norms[norms == 0] = 1.0
        Xn = X / norms
        S = Xn @ Xn.T  # cosine similarity in [-1,1]
        # ensure numerical range
        S = np.clip(S, -1.0, 1.0)
        return 1.0 - S
    else:
        raise ValueError(f"Unknown metric '{metric}'. Use 'euclidean' or 'cosine'.")


def gw_distance_between_layers_from_graphs(G1: nx.Graph, G2: nx.Graph) -> float:
    nodes1 = [str(n) for n in G1.nodes()]
    nodes2 = [str(n) for n in G2.nodes()]
    C1 = _all_pairs_shortest_path_matrix(G1, nodes1)
    C2 = _all_pairs_shortest_path_matrix(G2, nodes2)
    p = np.ones(len(nodes1)) / max(len(nodes1), 1)
    q = np.ones(len(nodes2)) / max(len(nodes2), 1)
    gw2 = gromov_wasserstein2(C1, C2, p, q, loss_function="square_loss")
    return float(np.sqrt(max(gw2, 0.0)))


def gw_distance_between_layers_from_embeddings(emb1: pd.DataFrame, emb2: pd.DataFrame, metric: str = 'euclidean') -> float:
    # Keep node ordering stable but arbitrary
    emb1 = emb1.sort_index()
    emb2 = emb2.sort_index()
    C1 = _pairwise_distance_matrix_from_embeddings(emb1, metric=metric)
    C2 = _pairwise_distance_matrix_from_embeddings(emb2, metric=metric)
    n1, n2 = C1.shape[0], C2.shape[0]
    p = np.ones(n1) / max(n1, 1)
    q = np.ones(n2) / max(n2, 1)
    gw2 = gromov_wasserstein2(C1, C2, p, q, loss_function="square_loss")
    return float(np.sqrt(max(gw2, 0.0)))



Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
df = read_edges(CSV_PATH)
graphs = build_layer_graphs(df)

# Embeddings per layer (store in dict)
layer_names = sorted(graphs.keys(), key=lambda x: (str(x)))
embeddings: Dict[str, pd.DataFrame] = {}
for layer in layer_names:
    emb = compute_node2vec_embeddings(
        graphs[layer],
        dimensions=DIMENSIONS,
        walk_length=WALK_LENGTH,
        num_walks=NUM_WALKS,
        p=P,
        q=Q,
        window=WINDOW,
        workers=WORKERS,
        seed=SEED,
    )
    embeddings[layer] = emb
    emb_path = os.path.join(OUT_DIR, f"embeddings_layer_{layer}.csv")
    emb.to_csv(emb_path)
    print(f"Saved embeddings for layer {layer} -> {emb_path}")

# Pairwise GW distances between layers
L = len(layer_names)
D = np.zeros((L, L), dtype=float)
for i in range(L):
    for j in range(i, L):
        if GW_SPACE == 'embedding':
            d = gw_distance_between_layers_from_embeddings(
                embeddings[layer_names[i]], embeddings[layer_names[j]], metric=GW_EMB_METRIC
            )
        elif GW_SPACE == 'shortest_path':
            d = gw_distance_between_layers_from_graphs(
                graphs[layer_names[i]], graphs[layer_names[j]]
            )
        else:
            raise ValueError("GW_SPACE must be 'embedding' or 'shortest_path'")
        D[i, j] = D[j, i] = d
        print(f"GW distance({layer_names[i]}, {layer_names[j]}) = {d:.6f}")

dist_df = pd.DataFrame(D, index=layer_names, columns=layer_names)
dist_csv = os.path.join(OUT_DIR, "gw_distances.csv")
dist_df.to_csv(dist_csv)
print(f"Saved GW distance matrix -> {dist_csv}")

Saved embeddings for layer 0 -> out/embeddings_layer_0.csv
Saved embeddings for layer 1 -> out/embeddings_layer_1.csv


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Saved embeddings for layer 2 -> out/embeddings_layer_2.csv


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Saved embeddings for layer 3 -> out/embeddings_layer_3.csv
Saved embeddings for layer 4 -> out/embeddings_layer_4.csv
GW distance(0, 0) = 0.000000
GW distance(0, 1) = 0.674594
GW distance(0, 2) = 0.677660
GW distance(0, 3) = 0.641303
GW distance(0, 4) = 0.620872
GW distance(1, 1) = 0.000000
GW distance(1, 2) = 0.319740
GW distance(1, 3) = 0.320457
GW distance(1, 4) = 0.343885
GW distance(2, 2) = 0.000000
GW distance(2, 3) = 0.354276
GW distance(2, 4) = 0.344384
GW distance(3, 3) = 0.000000
GW distance(3, 4) = 0.330284
GW distance(4, 4) = 0.000000
Saved GW distance matrix -> out/gw_distances.csv


In [20]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import networkx as nx

# ========= CONFIG =========
CSV_PATH = "sync_data/test.csv"
# node2vec
DIMENSIONS = 32
WALK_LENGTH = 80
NUM_WALKS = 10
P = 1.0
Q = 1.0
WINDOW = 10
WORKERS = 1
SEED = 42
# GW & interlayer linking
GW_EMB_METRIC = "euclidean"       # 'euclidean' or 'cosine'
TOP_K = 5                         # 每个源节点跨层只保留前 K 个目标
MIN_WEIGHT = 1e-3                 # 小于阈值的跨层边丢弃
ALPHA = 2.0                       # 层间全局缩放 Omega = exp(-ALPHA * GW)
INTRA_EDGE_WEIGHT = 1.0           # 层内边权（CSV 若无权重，就用常数 1）

# ========= deps =========
from node2vec import Node2Vec
from ot.gromov import gromov_wasserstein, gromov_wasserstein2
from infomap import Infomap


# ---------- data / graphs ----------
def read_edges(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    req = {"u", "v", "layer"}
    miss = req - set(df.columns)
    if miss:
        raise ValueError(f"CSV missing required columns: {sorted(miss)}")
    # 统一为字符串 id
    df["u"] = df["u"].astype(str)
    df["v"] = df["v"].astype(str)
    df["layer"] = df["layer"].astype(str)
    return df

def build_layer_graphs(df: pd.DataFrame) -> Dict[str, nx.Graph]:
    graphs: Dict[str, nx.Graph] = {}
    for layer, gdf in df.groupby("layer"):
        G = nx.Graph()
        if "w" in gdf.columns:
            edges = [(u, v, float(w)) for u, v, w in gdf[["u", "v", "w"]].itertuples(index=False)]
            G.add_weighted_edges_from(edges)
        else:
            G.add_edges_from(gdf[["u", "v"]].itertuples(index=False, name=None))
            # 给无权图统一权重 1
            nx.set_edge_attributes(G, INTRA_EDGE_WEIGHT, "weight")
        # 确保孤立点也在图里（如果 CSV 有只出现一次的端点）
        for col in ["u", "v"]:
            for n in gdf[col].unique():
                if n not in G:
                    G.add_node(n)
        graphs[layer] = G
    return graphs


# ---------- embeddings ----------
def compute_node2vec_embeddings(
    G: nx.Graph,
    dimensions=64, walk_length=80, num_walks=10, p=1.0, q=1.0,
    window=10, workers=1, seed=42
) -> pd.DataFrame:
    if len(G) == 0:
        raise ValueError("Empty graph.")
    def _fit(n_jobs):
        n2v = Node2Vec(G, dimensions=dimensions, walk_length=walk_length,
                       num_walks=num_walks, p=p, q=q, workers=n_jobs,
                       seed=seed, quiet=True)
        return n2v.fit(window=window, min_count=1, batch_words=4)
    try:
        model = _fit(workers)
    except Exception as e:
        print(f"[WARN] node2vec workers={workers} failed: {e}. Fallback to workers=1.")
        model = _fit(1)
    nodes = [str(n) for n in G.nodes()]
    X = np.vstack([model.wv[n] for n in nodes])
    emb = pd.DataFrame(X, index=nodes, columns=[f"f{i}" for i in range(X.shape[1])])
    emb.index.name = "node"
    return emb

def pairwise_from_embeddings(emb: pd.DataFrame, metric="euclidean") -> np.ndarray:
    X = emb.values.astype(float)
    if metric == "euclidean":
        s = np.sum(X * X, axis=1, keepdims=True)
        D2 = s + s.T - 2 * (X @ X.T)
        np.maximum(D2, 0.0, out=D2)
        return np.sqrt(D2, dtype=float)
    elif metric == "cosine":
        n = np.linalg.norm(X, axis=1, keepdims=True)
        n[n == 0] = 1.0
        Xn = X / n
        S = np.clip(Xn @ Xn.T, -1.0, 1.0)
        return 1.0 - S
    else:
        raise ValueError("metric must be 'euclidean' or 'cosine'.")

def gw_coupling_and_distance(emb1: pd.DataFrame, emb2: pd.DataFrame, metric="euclidean"):
    emb1 = emb1.sort_index()
    emb2 = emb2.sort_index()
    nodes1 = emb1.index.to_list()
    nodes2 = emb2.index.to_list()
    C1 = pairwise_from_embeddings(emb1, metric=metric)
    C2 = pairwise_from_embeddings(emb2, metric=metric)
    n1, n2 = C1.shape[0], C2.shape[0]
    p = np.ones(n1) / max(n1, 1)
    q = np.ones(n2) / max(n2, 1)
    # π（最优耦合，带熵正则版可用 ot.gromov.entropic_gromov_wasserstein）
    pi = gromov_wasserstein(C1, C2, p, q, loss_fun="square_loss")
    # GW^2
    gw2 = gromov_wasserstein2(C1, C2, p, q, loss_function="square_loss")
    gw = float(np.sqrt(max(gw2, 0.0)))
    return pi, nodes1, nodes2, gw

def state_id(node: str, layer: str) -> str:
    # 选择一个不太可能出现在节点/层名里的分隔符
    return f"{node}|{layer}"
def build_infomap_with_gw(
    graphs: Dict[str, nx.Graph],
    embeddings: Dict[str, pd.DataFrame],
    layer_order: List[str],
    top_k: int = 5,
    min_weight: float = 1e-3,
    alpha: float = 2.0,
    emb_metric: str = "euclidean",
    intra_weight_from_attr: str = "weight"
) -> Tuple[Infomap, Dict[int, Tuple[str, str]]]:
    """
    构造扁平化“状态节点图”，节点是 (node, layer) 的组合映射为 int。
    返回 (Infomap实例, id2state)，其中 id2state[i] = (node, layer)
    """

    # ----------- 1️⃣ 建立全局状态节点映射 -----------
    state2id = {}
    id2state = {}
    next_id = 1

    # 遍历所有层的所有节点，创建唯一的整数ID
    for layer in layer_order:
        for n in graphs[layer].nodes():
            key = (str(n), str(layer))
            state2id[key] = next_id
            id2state[next_id] = key
            next_id += 1

    im = Infomap("--two-level --silent")

    # ----------- 2️⃣ 层内边 -----------
    for layer in layer_order:
        G = graphs[layer]
        for u, v, d in G.edges(data=True):
            w = float(d.get(intra_weight_from_attr, INTRA_EDGE_WEIGHT))
            uid = state2id[(str(u), str(layer))]
            vid = state2id[(str(v), str(layer))]
            im.addLink(int(uid), int(vid), float(w))

        # 如果整层没有边，加极小自环，保证节点可见
        if G.number_of_edges() == 0:
            for n in G.nodes():
                nid = state2id[(str(n), str(layer))]
                im.addLink(int(nid), int(nid), 1e-12)

    # ----------- 3️⃣ 层间边 (GW 耦合) -----------
    for t in range(len(layer_order) - 1):
        la, lb = layer_order[t], layer_order[t + 1]
        emb_a, emb_b = embeddings[la].sort_index(), embeddings[lb].sort_index()
        pi, nodes_a, nodes_b, gw = gw_coupling_and_distance(emb_a, emb_b, metric=emb_metric)

        Omega = float(np.exp(-alpha * gw))  # 全局缩放
        row_sums = pi.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        P = pi / row_sums

        na, nb = P.shape
        for ia in range(na):
            k_use = min(top_k, nb) if top_k > 0 else nb
            idx = np.argpartition(-P[ia, :], kth=k_use - 1)[:k_use]
            idx = idx[np.argsort(-P[ia, idx])]

            src_name = nodes_a[ia]
            src_id = state2id[(str(src_name), str(la))]

            for jb in idx:
                w = float(P[ia, jb]) * Omega
                if w < min_weight:
                    continue
                dst_name = nodes_b[jb]
                dst_id = state2id[(str(dst_name), str(lb))]
                im.addLink(int(src_id), int(dst_id), float(w))

    return im, id2state


# ============ RUN ============
df = read_edges(CSV_PATH)
graphs = build_layer_graphs(df)
layer_names = sorted(graphs.keys(), key=lambda x: x)  # 你也可以自定义顺序

# 逐层嵌入
embeddings: Dict[str, pd.DataFrame] = {}
for layer in layer_names:
    embeddings[layer] = compute_node2vec_embeddings(
        graphs[layer],
        dimensions=DIMENSIONS, walk_length=WALK_LENGTH, num_walks=NUM_WALKS,
        p=P, q=Q, window=WINDOW, workers=WORKERS, seed=SEED
    )

    
im, id2state = build_infomap_with_gw(
    graphs, embeddings, layer_order=layer_names,
    top_k=TOP_K, min_weight=MIN_WEIGHT, alpha=ALPHA, emb_metric=GW_EMB_METRIC
)

im.run()

# 提取社区划分结果
partition = {n.node_id: n.module_id for n in im.tree if n.is_leaf}

# 映射回 (node, layer)
rows = []
for nid, com in partition.items():
    if nid in id2state:
        node, layer = id2state[nid]
        rows.append((node, layer, com))

assignments_df = pd.DataFrame(rows, columns=["node_id", "layer", "community"])
assignments_df.to_csv("out/infomap_state_nodes.csv", index=False)
print(f"Saved community assignments -> out/infomap_state_nodes.csv ({len(rows)} nodes)")


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Saved community assignments -> out/infomap_state_nodes.csv (1820 nodes)


In [21]:
import measurement

from measurement import *

compute_layerwise_ami_avg("test","sync_data",output_dir="output")

[0.6052790034077675, 0.9344613973053375, 0.8789571703502499, 0.9038561123671298, 0.9164749858448414]


0.8478057338550652